# Building a neural network FROM SCRATCH
This notebook follows this video: [Link](https://www.youtube.com/watch?v=w8yWXqWQYmU)

## The Math
![](../media/mnst-ffnn-math.png)

### Matrix size analysis
Our NN will have a simple two-layer architecture. Input layer $A^{[0]}$ will have 784 units corresponding to the 784 pixels in each 28x28 input image. A hidden layer $A^{[1]}$ will have 10 units with ReLU activation, and finally our output layer $A^{[2]}$ will have 10 units corresponding to the ten digit classes with softmax activation.

**Forward propagation**

$$Z^{[1]} = W^{[1]} X + b^{[1]}$$
$$A^{[1]} = g_{\text{ReLU}}(Z^{[1]}))$$
$$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$$
$$A^{[2]} = g_{\text{softmax}}(Z^{[2]})$$

**Backward propagation**

$$dZ^{[2]} = A^{[2]} - Y$$ 
$$dW^{[2]} = \frac{1}{m} dZ^{[2]} A^{[1]T}$$
$$dB^{[2]} = \frac{1}{m} \Sigma {dZ^{[2]}}$$
$$dZ^{[1]} = W^{[2]T} dZ^{[2]} .* g^{[1]\prime} (z^{[1]})$$
$$dW^{[1]} = \frac{1}{m} dZ^{[1]} A^{[0]T}$$
$$dB^{[1]} = \frac{1}{m} \Sigma {dZ^{[1]}}$$

**Parameter updates**

$$W^{[2]} := W^{[2]} - \alpha dW^{[2]}$$
$$b^{[2]} := b^{[2]} - \alpha db^{[2]}$$
$$W^{[1]} := W^{[1]} - \alpha dW^{[1]}$$
$$b^{[1]} := b^{[1]} - \alpha db^{[1]}$$

**Vars and shapes**

Forward prop

- $A^{[0]} = X$: 784 x m
- $Z^{[1]} \sim A^{[1]}$: 10 x m
- $W^{[1]}$: 10 x 784 (as $W^{[1]} A^{[0]} \sim Z^{[1]}$)
- $B^{[1]}$: 10 x 1
- $Z^{[2]} \sim A^{[2]}$: 10 x m
- $W^{[2]}$: 10 x 10 (as $W^{[2]} A^{[1]} \sim Z^{[2]}$)
- $B^{[2]}$: 10 x 1

Backprop

- $dZ^{[2]}$: 10 x m ($~A^{[2]}$)
- $dW^{[2]}$: 10 x 10
- $dB^{[2]}$: 10 x 1
- $dZ^{[1]}$: 10 x m ($~A^{[1]}$)
- $dW^{[1]}$: 10 x 10
- $dB^{[1]}$: 10 x 1

Note that here is no specific loss function, its just the difference between predicted and true labels: $dZ^{[2]} = A^{[2]} - Y$

## The Code

### Imports

In [1]:
# Imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

### Data

In [2]:
# Data Imports
train_data_df = pd.read_csv("data/mnist_train.csv")
dev_data_df = pd.read_csv("data/mnist_test.csv")

In [3]:
train_data_df.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
dev_data_df.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Create np array from data
train_data = np.array(train_data_df)
dev_data = np.array(dev_data_df)

# Shuffles training data
np.random.shuffle(train_data) # Shuffles at image level, not pixel level!

In [6]:
# Note: Labels are included in this data an we need to separate it. The images are
# 28x28 so we need to remove first one column.
print(train_data.shape)
print(dev_data.shape)

(60000, 785)
(10000, 785)


In [7]:
# Remove labels from data
X_train = train_data[:,1:].T # All rows but avoid first column since its the labels
Y_train = train_data[:,0].T # All rows but keep just first column since its the labels

X_dev = dev_data[:,1:].T
Y_dev = dev_data[:,0].T

In [8]:
# Normalizing
X_train = X_train / 255
X_dev = X_dev / 255

In [9]:
print(X_train.shape)
print(Y_train.shape)

print(X_dev.shape)
print(Y_dev.shape)

(784, 60000)
(60000,)
(784, 10000)
(10000,)


In [10]:
m, n = X_train.shape

### Training

In [11]:
def init_params():
    '''
    Initialize random weights and biases
    '''
    W1 = np.random.rand(10, 784) - 0.5 # Set of weights for first hidden state with 10 neurons. Each pixel is connected to 10 neurons and each needs to have its sets of weights.
    b1 = np.random.rand(10, 1) - 0.5 # Bias for first hidden state with 10 neurons. Note how bias is dependent on the number of neurons and not the number of input pixels.
    W2 = np.random.rand(10, 10) - 0.5 # Dependent on previous layer W1
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

In [12]:
def ReLU(Z):
    result = np.maximum(Z, 0) # This is element wise. Goes through each element in Z and return appropriately.
    assert (result.shape == Z.shape)
    return result


def softmax(Z):
    result = np.exp(Z) / sum(np.exp(Z)) #  Returns normalized values
    assert (result.shape == Z.shape)
    return result

In [25]:
def forward_prop(W1, b1, W2, b2, X, flag="Test"):
    '''
    Computes forward propagation of NN on batch X of samples
    '''
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)

    # Assert shapes only on training. Debugging purposes only
    if flag == 'Train':
        assert (Z1.shape == (10, n))
        assert (A1.shape == (10, n))
        assert (Z2.shape == (10, n))
        assert (A2.shape == (10, n))

    return Z1, A1, Z2, A2


In [14]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, 10))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [15]:
def deriv_ReLU(Z):
    return Z > 0 # Will return 1 or 0 array. True converts to 1 and False to 0

In [16]:
def back_prop(Z1, A1, Z2, A2, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y # Predictions - Labels. Is this a simplification??? What loss function are we using?
    dW2 = (1 / n) * dZ2.dot(A1.T)
    db2 = ((1 / n) * np.sum(dZ2, axis=1)).reshape((10, 1))
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1) 
    dW1 = (1 / n) * dZ1.dot(X.T)
    db1 = ((1 / n) * np.sum(dZ1, axis=1)).reshape((10, 1))
    
    return dW1, db1, dW2, db2 

In [17]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha): # alpha is the learning rate
    W1 = W1 - alpha * dW1 # PROBLEM: W1 is 10x784 and dW1 is 10x10. These are the expected shapes: (10, 784) (10, 784)
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2


In [18]:
def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

In [19]:
def get_predictions(A2):
    return np.argmax(A2, 0)

In [20]:
def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2 # Our trained model!

In [22]:
# Run our model
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 100, 0.1)

Iteration:  0
0.09206666666666667
Iteration:  10
0.19085
Iteration:  20
0.28786666666666666
Iteration:  30
0.38581666666666664
Iteration:  40
0.44808333333333333
Iteration:  50
0.49128333333333335
Iteration:  60
0.5244666666666666
Iteration:  70
0.5515333333333333
Iteration:  80
0.5753833333333334
Iteration:  90
0.5954833333333334


For debugging I went straight to the source: [Link](https://www.kaggle.com/code/fcanova/simple-mnist-nn-from-scratch-numpy-no-tf-keras/edit)

### Evaluating

In [29]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X, "Evaluate")
    predictions = get_predictions(A2)
    return predictions

In [31]:
dev_pred = make_predictions(X_dev, W1, b1, W2, b2)
print(get_accuracy(dev_pred, Y_dev))

0.6199


In [ ]:
# Lets analyze how the model does with each of our classes. For that lets analyze class by class

## Notes

### Normalizing
Note that if we dont normalize the data, accuracies during training are the following: 
- Iteration:  0: **0.08261666666666667**
- Iteration:  10: **0.09871666666666666**
- Iteration:  20: **0.09871666666666666**
- Iteration:  30: **0.09871666666666666**
- Iteration:  40: **0.09871666666666666**
- Iteration:  50: **0.09871666666666666**
- Iteration:  60: **0.09871666666666666**
- Iteration:  70: **0.09871666666666666**
- Iteration:  80: **0.09871666666666666**
- Iteration:  90: **0.09871666666666666**

The reason for such low accuracies in this NN achitecture has to do with overflow issues when applying the softmax funcion. For some values of Z2, after applying the softmax function we get silent overflow errors while calculating ```np.exp(Z) / np.sum(np.exp(Z))``` which defaults to 0 effectively converting our Z2 into almost an all zero matrix.




### Training Batches
In this architecture we feed all our training samples multiple times for training. Note that this is possible only if all our training data is able to fit into memory at once. In most cases this is not the case and the only viable alternative we have left is to use batch training.

## Moving Forward
### Model
- Explore more layers
- Try different learning rates
- Try different learning rates as we are learning (modify alpha in iterations?)
- Try different batch sizes (using batch normalization)
- Expore regularization techniques
- Normalizing inputs
- Use a loss function (up to now you are only substracting predictions to true labels!)
- Normalize Data
- Use more data? What about synthetic data?

### Ops
- The data is too heavy to push to repositories like github. Could this be sorted by preloading the data to the docker container on build time? That way, when a new user wants to run the code, the data is preloaded inside the dev environment.
- Data Science teams are the ones that iterate through this code and make this model more effective. How can you provide a way to give them traceability thoughout experiments with a tool like W&B?